This notebook reads in the 2016, 2017, and 2018 Q1 [Origin and Destination Survey DB1BCoupon data](https://www.transtats.bts.gov/Fields.asp) which provides quarterly 10% samples of all passenger tickets -- this particular set of data gives each enplanement (or leg) on a given itinerary and will provide fare class of given passengers (business/first/economy). I am using this data to engineer a feature for the typical business/first class passenger ratio on a given origin-destination pair (on a single flight) for a given quarter -- the thinking being that more passengers in this fare class indicate more air travel in congested time periods.

In [1]:
import pandas as pd
import numpy as np

import pickle

Read in the data for 2016 and 2017.

In [2]:
db1b_coup_tr = pd.DataFrame()
for qtr in [1,2,3,4]:
    tmp = pd.read_csv('DB1BCoupon/Origin_and_Destination_Survey_DB1BCoupon_2016_' + str(qtr) + '.csv')
    db1b_coup_tr = pd.concat([db1b_coup_tr, tmp], axis=0)
    print(qtr)

1
2
3
4


In [3]:
for qtr in [1,2,3,4]:
    tmp = pd.read_csv('DB1BCoupon/Origin_and_Destination_Survey_DB1BCoupon_2017_' + str(qtr) + '.csv')
    db1b_coup_tr = pd.concat([db1b_coup_tr, tmp], axis=0)
    print(qtr)

1
2
3
4


Now for 2018

In [4]:
tmp = pd.read_csv('DB1BCoupon/Origin_and_Destination_Survey_DB1BCoupon_2018_1.csv')
db1b_coup_tr = pd.concat([db1b_coup_tr, tmp], axis=0)

Keep only flights with the origin airports we are evaluating.

In [5]:
nyc_airports = ['EWR', 'JFK', 'LGA']

In [6]:
db1b_coup_tr = db1b_coup_tr[db1b_coup_tr['Origin'].isin(nyc_airports)]

In [7]:
db1b_coup_tr.shape

(4115954, 37)

Now load in the AOTP data we have saved to get the destination airports we are evaluating, and then keep only flights in the DB1B Coupon data for these destination airports.

In [8]:
with open('aotp_nyc_new.pkl', 'rb') as file1:
    aotp_data = pickle.load(file1)

In [9]:
dests = list(aotp_data['Dest'].unique())

In [10]:
len(dests)

57

In [11]:
del(aotp_data)

In [12]:
db1b_coup_tr = db1b_coup_tr[db1b_coup_tr['Dest'].isin(dests)]

In [13]:
db1b_coup_tr.shape

(3634831, 37)

In [14]:
db1b_coup_tr.columns.values

array(['ItinID', 'MktID', 'SeqNum', 'Coupons', 'Year', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'Quarter', 'Origin',
       'OriginCountry', 'OriginStateFips', 'OriginState',
       'OriginStateName', 'OriginWac', 'DestAirportID',
       'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCountry',
       'DestStateFips', 'DestState', 'DestStateName', 'DestWac', 'Break',
       'CouponType', 'TkCarrier', 'OpCarrier', 'RPCarrier', 'Passengers',
       'FareClass', 'Distance', 'DistanceGroup', 'Gateway', 'ItinGeoType',
       'CouponGeoType', 'Unnamed: 36'], dtype=object)

Keep only selected columns in the DB1B Coupon data and save.

In [15]:
db1b_coup_tr = db1b_coup_tr[['ItinID', 'MktID', 'Year', 'Quarter', 'Origin', 'Dest', 'RPCarrier', 'Passengers', 'FareClass']]

In [16]:
db1b_coup_tr.shape

(3634831, 9)

In [17]:
with open('db1b_coup.pkl', 'wb') as file:
    pickle.dump(db1b_coup_tr, file)